In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import openpyxl
import warnings
warnings.filterwarnings('ignore')  # 防止警告文件的包
%matplotlib inline
from time import time

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split  # 将数据集分成测试集和训练集
from imblearn.over_sampling import ADASYN

from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB  # naive bayesian
from sklearn.ensemble import RandomForestClassifier, VotingClassifier  
from sklearn.svm import SVC  ## 支持向量机分类模型

In [ ]:
#读入文件
match = pd.read_csv('../input/european-cup2/International football results from 1872 to 2021.csv')
#team = pd.read_csv('EuroCup21 Teams.csv')
#match
match.columns
match.columns.values
match.shape

In [ ]:
#删除无用信息
match2 = match.drop(['date','city'],1)
match2.head()

# 加工信息

In [ ]:
#分出测试集
test = match2[(match2['home_score'].isna())]
match3 = match2[(match2['home_score']>=0)]
match3.tail()
#test

In [ ]:
#算出净胜球
match3['result'] = match3['home_score'] - match3['away_score'] + 0
#match3['result'] = pd.to_numeric(match3['result'])
match3['result']
#match3['result'].unique()

In [ ]:
#将净胜球转化为输赢
for i in range(42329):
    if match3.loc[i,'result']>0:
        match3.loc[i,'result2']=1.0
    if match3.loc[i,'result']==0:
        match3.loc[i,'result2']=0.0
    if match3.loc[i,'result']<0:
        match3.loc[i,'result2']=-1.0
#match3['result2'].unique()
match3['result2']

In [ ]:
#用0,1表示是否中立
print(match3['neutral'].unique())
match3['neutral'] = match3['neutral'].map({True:1.0,False:0.0})
print(match3['neutral'].unique())
match3.head()

# 开始将文字变为数字

In [ ]:
match4 = match3

In [ ]:
#替换tournament
tour = match4['tournament'].unique()
print(tour)
tour_num = len(tour)
print(tour_num)

def replace_tournament(x):
    for i in range(tour_num):
        x = x.replace(tour[i],i)
    return x

match4['tournament'] = replace_tournament(match4['tournament'])

In [ ]:
#替换 away_team，home_team，country
a = match4['country']
b = match4['home_team']
c = match4['away_team']
team = pd.concat([a,b,c], 0,sort=False)
print(team)
teams = team.unique()
#print(teams)

team_num = len(teams)
print(team_num)

def replace_country(x):
    for i in range(team_num):
        x = x.replace(teams[i],i)
    return x

match4['home_team'] = replace_country(match4['home_team'])
match4['away_team'] = replace_country(match4['away_team'])
match4['country'] = replace_country(match4['country'])

In [ ]:
#检查
#match4['tournament'].unique()
#match4['country'].unique()
match4
#match4.shape

In [ ]:
#删去多余的列
match5 = match4.drop(['home_score','away_score','result'],1)
match5

# 开始训练

In [ ]:
#取出结果
match_y = match5.pop('result2')

In [ ]:
#分出训练集和验证集
np.random.seed(111)
train_x, valid_x, train_y, valid_y = train_test_split(match5, match_y, test_size=0.2)
valid_x

In [ ]:
#基准
valid_bench = np.ones(valid_y.shape)
valid_conf = confusion_matrix(valid_y,valid_bench)
print(valid_conf)

In [ ]:
#定义判断标准
def acc(conf):
    return np.sum(conf[:,0] * [1,-1,-1] + conf[:,1] * [-1,1,-1] + conf[:,2] * [-1,-1,1])
print(acc(valid_conf))

In [ ]:
#重抽样
m_resample = ADASYN()
print(train_y.__len__())
train_x2,train_y2 = m_resample.fit_resample(train_x,train_y)
print(train_y2.__len__())
#train_x
train_x2

# 调包

In [ ]:
#1
m = BernoulliNB()

m.fit(train_x, train_y)
valid_pred = m.predict(valid_x)
valid_conf = confusion_matrix(valid_y, valid_pred)
print(valid_conf)
print(acc(valid_conf))

In [ ]:
#1’
# m = BernoulliNB()

# m.fit(train_x2, train_y2)
# valid_pred = m.predict(valid_x)
# valid_conf = confusion_matrix(valid_y, valid_pred)
# print(valid_conf)
# print(acc(valid_conf))

In [ ]:
#2
m = GaussianNB()

m.fit(train_x, train_y)
valid_pred = m.predict(valid_x)
valid_conf = confusion_matrix(valid_y, valid_pred)
print(valid_conf)
print(acc(valid_conf))

In [ ]:
#2‘
# m = GaussianNB()

# m.fit(train_x2, train_y2)
# valid_pred = m.predict(valid_x)
# valid_conf = confusion_matrix(valid_y, valid_pred)
# print(valid_conf)
# print(acc(valid_conf))

In [ ]:
#3
t0 = time()

m = MLPClassifier(hidden_layer_sizes =(64,32,16))

m.fit(train_x, train_y)
valid_pred = m.predict(valid_x)
valid_conf = confusion_matrix(valid_y, valid_pred)
print(valid_conf)
print(acc(valid_conf))

t1 = time()
print('时长',t1-t0)

In [ ]:
#3’
# m = MLPClassifier(hidden_layer_sizes =(64,32,16))

# m.fit(train_x2, train_y2)
# valid_pred = m.predict(valid_x)
# valid_conf = confusion_matrix(valid_y, valid_pred)
# print(valid_conf)
# print(acc(valid_conf))

In [ ]:
#4
m = DecisionTreeClassifier()

m.fit(train_x, train_y)
valid_pred = m.predict(valid_x)
valid_conf = confusion_matrix(valid_y, valid_pred)
print(valid_conf)
print(acc(valid_conf))

In [ ]:
#5
t0 = time()

m = SVC()

m.fit(train_x, train_y)
valid_pred = m.predict(valid_x)
valid_conf = confusion_matrix(valid_y, valid_pred)
print(valid_conf)
print(acc(valid_conf))

t1 = time()
print('时长',t1-t0)

In [ ]:
#6
m = RandomForestClassifier(n_estimators=100)

m.fit(train_x, train_y)
valid_pred = m.predict(valid_x)
valid_conf = confusion_matrix(valid_y, valid_pred)
print(valid_conf)
print(acc(valid_conf))

In [ ]:
# 7
m1 = BernoulliNB()
m2 = GaussianNB()

m = VotingClassifier(estimators=[('BernoulliNB',m1),('G',m2)],voting='hard')
m.fit(train_x,train_y)
valid_pred = m.predict(valid_x)
valid_p_conf = confusion_matrix(valid_y, valid_pred)
print(valid_p_conf)
print(acc(valid_p_conf))

In [ ]:
# 8
m1 = MLPClassifier(hidden_layer_sizes =(32,16,8,4))
m2 = DecisionTreeClassifier()

m = VotingClassifier(estimators=[('NN',m1),('Decision',m2)],voting='hard')
m.fit(train_x,train_y)
valid_pred = m.predict(valid_x)
valid_p_conf = confusion_matrix(valid_y, valid_pred)
print(valid_p_conf)
print(acc(valid_p_conf))

In [ ]:
#9
m1 = RandomForestClassifier(n_estimators=100)
m2 = DecisionTreeClassifier()
m3 = MLPClassifier(hidden_layer_sizes =(32,16,8,4))

m = VotingClassifier(estimators=[('B',m1),('DC',m2),('NN',m3)],voting='hard')
m.fit(train_x,train_y)
valid_pred = m.predict(valid_x)
valid_p_conf = confusion_matrix(valid_y, valid_pred)
print(valid_p_conf)
print(acc(valid_p_conf))

In [ ]:
#10
m1 = GaussianNB()
m2 = SVC()
m3 = BernoulliNB()

m = VotingClassifier(estimators=[('G',m1),('SVC',m2),('NN',m3)],voting='hard')
m.fit(train_x,train_y)
valid_pred = m.predict(valid_x)
valid_p_conf = confusion_matrix(valid_y, valid_pred)
print(valid_p_conf)
print(acc(valid_p_conf))

In [ ]:
#11
m1 = BernoulliNB()
m2 = DecisionTreeClassifier()
m3 = MLPClassifier(hidden_layer_sizes =(32,16,8,4))
m4 = GaussianNB()

m_f = VotingClassifier(estimators=[('B',m1),('DC',m2),('MLP',m3),('G',m4)],voting='hard')
m_f.fit(train_x,train_y)
valid_pred = m_f.predict(valid_x)
valid_p_conf = confusion_matrix(valid_y, valid_pred)
print(valid_p_conf)
print(acc(valid_p_conf))

In [ ]:
#12
m1 = BernoulliNB()
m2 = DecisionTreeClassifier()
m3 = MLPClassifier(hidden_layer_sizes =(32,16,8,4))
m4 = GaussianNB()
m5 = RandomForestClassifier(n_estimators=100)

m = VotingClassifier(estimators=[('B',m1),('DC',m2),('MLP',m3),('G',m4),('RFC',m5)],voting='hard')
m.fit(train_x,train_y)
valid_pred = m.predict(valid_x)
valid_p_conf = confusion_matrix(valid_y, valid_pred)
print(valid_p_conf)
print(acc(valid_p_conf))

# 输出结果

In [ ]:
# 处理测试集
test_x = test.drop(['home_score','away_score'],1)
test_x['home_team'] = replace_country(test_x['home_team'])
test_x['away_team'] = replace_country(test_x['away_team'])
test_x['country'] = replace_country(test_x['country'])
test_x['neutral'] = test_x['neutral'].map({True:1.0,False:0.0})
test_x['tournament'] = replace_tournament(test_x['tournament'])

for i in test_x.keys():
    test_x[i].fillna(0.0,inplace=True)

test_x

In [ ]:
#提交结果
euro_submit= test_x[['home_team']].copy()
euro_submit['away_team'] = test_x['away_team']
euro_submit['tournament'] = test_x['tournament']
euro_submit['prediction'] = m_f.predict(test_x)
#euro_submit.to_excel('../output/kaggle/working/euro_submit.xlsx', index=False)
euro_submit

In [ ]:
#将数字转换回文字
def place_tournament(x):
    for i in range(tour_num):
        x = x.replace(i,tour[i])
    return x

def place_country(x):
    for i in range(team_num):
        x = x.replace(i,teams[i])
    return x

euro_submit['home_team'] = place_country(euro_submit['home_team'])
euro_submit['away_team'] = place_country(euro_submit['away_team'])
euro_submit['tournament'] = place_tournament(euro_submit['tournament'])
euro_submit

#数字为预测的每场比赛的胜负情况
#1表示主场队胜利，0表示平局，-1表示客场队胜利